In [1]:
!pip install plotly
!pip install dash
!pip install jupyter_dash
!pip install IPython
!pip install numpy
!pip install pandas
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.6 MB/s eta 0:00:00


In [2]:
from jupyter_dash import JupyterDash
from urllib.request import urlopen
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, ctx
from dash.dependencies import Input, Output


In [24]:
import requests
def load_data():
    urls = {
        "2020": "https://data.gov.il/api/3/action/datastore_search?resource_id=520597e3-6003-4247-9634-0ae85434b971",
        "2021": "https://data.gov.il/api/3/action/datastore_search?resource_id=3f71fd16-25b8-4cfe-8661-e6199db3eb12",
        "2022": "https://data.gov.il/api/3/action/datastore_search?resource_id=a59f3e9e-a7fe-4375-97d0-76cea68382c1",
        "2023": "https://data.gov.il/api/3/action/datastore_search?resource_id=32aacfc9-3524-4fba-a282-3af052380244",
        "2024": "https://data.gov.il/api/3/action/datastore_search?resource_id=5fc13c50-b6f3-4712-b831-a75e0f91a17e",
    }
    data_frames = []
    for year, url in urls.items():
        response = requests.get(url)
        data = response.json()
        records = data['result']['records']
        df = pd.DataFrame(records)
        df['Year'] = int(year)  # Add year column
        df["Category"] = df["StatisticGroup"].apply(categorize_statistic_group)
        df = df.dropna(subset=["Category"])
        df["ReversedStatisticGroup"] = df["Category"].apply(lambda x: x[::-1])
        data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

In [23]:
def categorize_statistic_group(stat_group):
    categories = {
        "עבירות פליליות כלליות": ['עבירות כלפי הרכוש', 'עבירות נגד גוף', 'עבירות נגד אדם', 'עבירות מין'],
        "עבירות מוסר וסדר ציבורי": ['עבירות כלפי המוסר', 'עבירות סדר ציבורי'],
        "עבירות ביטחון": ['עבירות בטחון'],
        "עבירות כלכליות ומנהליות": ['עבירות כלכליות', 'עבירות מנהליות', 'עבירות רשוי'],
        "עבירות תנועה": ['עבירות תנועה'],
        "עבירות מרמה": ['עבירות מרמה']
    }
    for category, types in categories.items():
        if stat_group in types:
            return category
    return None

In [25]:
df = load_data()
df

,_id,FictiveIDNumber,Year,Quarter,YeshuvKod,Yeshuv,PoliceDistrictKod,PoliceDistrict,PoliceMerhavKod,PoliceMerhav,...,municipalKod,municipalName,StatisticAreaKod,StatisticArea,StatisticGroupKod,StatisticGroup,StatisticTypeKod,StatisticType,Category,ReversedStatisticGroup
0,1,D681A6DC3CF151AA4846DF0893A1BDE7,2020,Q2,5000.0,תל אביב יפו,20000000.0,מחוז תא,21800000.0,מרחב איילון החדש תא,...,NaN,None,50000934.0,התקווה (מרכז),700,עבירות כלפי הרכוש,719,גרימת נזק לרכוש בזדון,עבירות פליליות כלליות,תויללכ תוילילפ תוריבע
1,2,CC7274FAB8CECA43AFBB5598A031C7D8,2020,Q2,7900.0,פתח תקווה,40000000.0,מחוז מרכז,41100000.0,מרחב שרון,...,NaN,None,79000433.0,קרית חזני (מערב),600,עבירות כלפי המוסר,609,סחריבוא ויצוא סמים,עבירות מוסר וסדר ציבורי,ירוביצ רדסו רסומ תוריבע
2,3,DD63A80B87771A39D856D709781E8DEE,2020,Q1,6600.0,חולון,20000000.0,מחוז תא,21800000.0,מרחב איילון החדש תא,...,NaN,None,66000413.0,נאות שושנים (דרום),700,עבירות כלפי הרכוש,715,גניבות אחרות,עבירות פליליות כלליות,תויללכ תוילילפ תוריבע
3,4,9F3E82C433BDF624F57407B1E190F95A,2020,Q3,5000.0,תל אביב יפו,20000000.0,מחוז תא,21700000.0,מרחב ירקון תא,...,NaN,None,50000515.0,מרכז תא (5),400,עבירות נגד גוף,404,(תקיפה (למעט עובדי ציבור,עבירות פליליות כלליות,תויללכ תוילילפ תוריבע
4,5,6406AAB8948F9D1641422A252DA3BD8D,2020,Q1,NaN,None,70000000.0,מחוז שי,71500000.0,מרחב שומרון שי,...,NaN,ישוב פלסטיני,NaN,None,200,עבירות סדר ציבורי,220,עבירות על חוק הכניסה לישראל,עבירות מוסר וסדר ציבורי,ירוביצ רדסו רסומ תוריבע
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159502,31996,90436EB14258AF1D12D77E823DC6B4C5,2024,Q2,5000.0,תל אביב יפו,20000000.0,מחוז תא,21700000.0,מרחב ירקון תא,...,NaN,None,50000114.0,תוכנית ל' (מרכז),800,עבירות מרמה,803,מרמה ועושק,עבירות מרמה,המרמ תוריבע
159503,31997,AEF086FF79FFD7AFD3C2D51E23A1F59C,2024,Q4,4000.0,חיפה,80000000.0,מחוז חוף,15005216.0,מרחב כרמל חוף,...,NaN,None,40000811.0,זיו,700,עבירות כלפי הרכוש,719,גרימת נזק לרכוש בזדון,עבירות פליליות כלליות,תויללכ תוילילפ תוריבע
159504,31998,4D1F4DB7F1F6F3A15C22A5E4E862386B,2024,Q2,NaN,None,70000000.0,מחוז שי,71400000.0,מרחב יהודה שי,...,NaN,ישוב פלסטיני,NaN,None,200,עבירות סדר ציבורי,200,עבירות סדר ציבורי,עבירות מוסר וסדר ציבורי,ירוביצ רדסו רסומ תוריבע
159505,31999,0E4FD63F4C4B2D9C3174AD687506E982,2024,Q1,7100.0,אשקלון,30000000.0,מחוז דרומי,31200000.0,מרחב לכיש,...,NaN,None,71000313.0,שיקמים (שמשון מזרח),800,עבירות מרמה,803,מרמה ועושק,עבירות מרמה,המרמ תוריבע


In [105]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
import seaborn as sns

In [153]:
app = dash.Dash(__name__)

In [154]:
y_max = df["StatisticType"].value_counts().max()
years = ["All Years (mean)"] + sorted(df["Year"].dropna().unique().astype(int).tolist())

# Layout for the app
app.layout = html.Div([
    html.H1("Crime Analysis Dashboard", style={"textAlign": "center"}),

    html.Div([
        html.Label("Select Year:", style={"textAlign": "center", "display": "block"}),
        dcc.Dropdown(
            id="year-dropdown",
            options=[{"label": str(year), "value": year} for year in years],
            value="All Years (mean)",
            style={"width": "200px", "margin": "0 auto"}
        ),
            dcc.Checklist(
            id="facet-checkbox",
            options=[{"label": "Split by Quarter", "value": "facet"}],
            value=[],
            style={"textAlign": "center", "marginTop": "10px"}
        )
    ], style={"width": "30%", "margin": "0 auto",  "textAlign": "center"}),

    html.Div(id="bar-chart-container", style= {"textAlign": "center"})
])

# Callback for updating the bar chart
@app.callback(
    Output("bar-chart-container", "children"),
    [Input("year-dropdown", "value"), Input("facet-checkbox", "value")]
)
def update_bar_chart(selected_year, facet_option):
    cats = df["ReversedStatisticGroup"].drop_duplicates().tolist()
    # Filter data based on selected year
    if selected_year == "All Years (mean)":
        filtered_data = df
        crime_counts = (filtered_data.groupby("ReversedStatisticGroup").size() / 5).reindex(cats, fill_value=0)

    else:
        filtered_data = df[df["Year"] == int(selected_year)]
        crime_counts = (filtered_data["ReversedStatisticGroup"].value_counts()).reindex(cats, fill_value=0)

    if "facet" in facet_option:
          # Create faceted bar chart by quarter
          grouped_data = (
              filtered_data.groupby(["ReversedStatisticGroup", "Quarter"]).size().reset_index(name="Counts")
          )
          fig, ax = plt.subplots(figsize=(12, 8))
          sns.barplot(
              data=grouped_data,
              x="ReversedStatisticGroup",
              y="Counts",
              hue="Quarter",
              palette="deep",
              ax=ax
          )
          ax.set_title("גוסו ןועבר יפל םיעשפה רפסמ", fontsize=16)
          ax.set_xlabel("עשפה גוס", fontsize=14)
          ax.set_ylabel("ןועבר", fontsize=14)
    else:
        # Create regular bar chart
        fig, ax = plt.subplots(figsize=(10, 6))
        crime_counts.plot(kind="bar", ax=ax, color="orange")
        ax.set_title("גוס יפל םיעשפה רפסמ", fontsize=16)
        ax.set_xlabel("עשפה גוס", fontsize=14)
        ax.set_ylabel("םיעשפה רפסמ", fontsize=14)
        ax.tick_params(axis="x", labelrotation=45)
        ax.set_ylim(1000, y_max)

    plt.tight_layout()

    # Convert plot to HTML image
    buffer = io.BytesIO()
    plt.savefig(buffer, format="png")
    buffer.seek(0)
    image_base64 = base64.b64encode(buffer.getvalue()).decode()
    buffer.close()
    plt.close(fig)

    return html.Img(src="data:image/png;base64," + image_base64)

# Run the app
# if __name__ == "__main__":
#     app.run_server()

In [155]:
from google.colab import output
output.serve_kernel_port_as_window(8050)

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

In [156]:
app.run_server( debug=False,dev_tools_ui=False,dev_tools_props_check=False)

<IPython.core.display.Javascript object>